
# HDA Flowsheet Costing


## Note

This tutorial will demonstrate adding costing to the two HDA examples, the basic [HDA with Flash](../../Tutorials/Basics/HDA_flowsheet_solution_testing.ipynb) and a comparison with the [HDA with Distillation](HDA_flowsheet_with_distillation_solution_testing.ipynb).


## Learning outcomes


- Import external pre-built steady-state flowsheets using the IDAES unit model library
- Define and add costing blocks using the IDAES Generic Costing Framework
- Fomulate and solve a process economics optimization problem
    - Defining an objective function
    - Setting variable bounds
    - Adding additional constraints 


## Problem Statement

Hydrodealkylation is a chemical reaction that often involves reacting
an aromatic hydrocarbon in the presence of hydrogen gas to form a
simpler aromatic hydrocarbon devoid of functional groups. In this
example, toluene will be reacted with hydrogen gas at high temperatures
 to form benzene via the following reaction:

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> → C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>**


This reaction is often accompanied by an equilibrium side reaction
which forms diphenyl, which we will neglect for this example.

This example is based on the 1967 AIChE Student Contest problem as
present by Douglas, J.M., Chemical  Design of Chemical Processes, 1988,
McGraw-Hill.

Users may refer to the prior examples linked at the top of this notebook for detailed process descriptions of the two HDA configurations. As before, the properties required for this module are defined in

- `hda_ideal_VLE.py`
- `idaes.generic_models.properties.activity_coeff_models.BTX_activity_coeff_VLE`
- `hda_reaction_kinetic.py`

Additionally, we will be importing externally-defined flowsheets for the two HDA configurations from

- `hda_flowsheets.py`

In [ ]:
from hda_flowsheets import hda_with_flash, hda_with_distillation

In [ ]:
m = hda_with_flash()

In [ ]:
n = hda_with_distillation()